In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, MultiPolygon

%matplotlib inline
import cartoframes

import json
    
cc = cartoframes.CartoContext()

import requests
import io

In [25]:
# TODO: Get shoreline clipped boundaries for block groups, maybe?
blk_grp = gpd.read_file('gz_2010_36_150_00_500k/gz_2010_36_150_00_500k.shp')

## 1. Get block groups only in the counties of interest

In [29]:
county_list =['005',
'047',
'061',
'081',
'085',
] ## Counties in NYC

blk_grp=blk_grp[blk_grp['COUNTY'].isin(county_list)]

## 2. Define dictionaries of ACS variable codes and their variable names

In [3]:
age_sex_vars = {'B01003_001E': 'Total Population',
                    "B01001_026E":"Female",
                    "B01001_002E":"Male",
                    "B01001_003E":"Male, Under 5 years",
                    "B01001_004E":"Male, 5 to 9 years",
                    "B01001_005E":"Male, 10 to 14 years",
                    "B01001_006E":"Male,s 15 to 17 years",
                    "B01001_007E":"Male, 18 and 19 years",
                    "B01001_008E":"Male, 20 years",
                    "B01001_009E":"Male, 21 years",
                    "B01001_010E":"Male, 22 to 24 years",
                    "B01001_011E":"Male, 25 to 29 years",
                    "B01001_012E":"Male, 30 to 34 years",
                    "B01001_013E":"Male, 35 to 39 years",
                    "B01001_014E":"Male, 40 to 44 years",
                    "B01001_015E":"Male, 45 to 49 years",
                    "B01001_016E":"Male, 50 to 54 years",
                    "B01001_017E":"Male, 55 to 59 years",
                    "B01001_018E":"Male, 60 and 61 years",
                    "B01001_019E":"Male, 62 to 64 years",
                    "B01001_020E":"Male, 65 and 66 years",
                    "B01001_021E":"Male, 67 to 69 years",
                    "B01001_022E":"Male, 70 to 74 years",
                    "B01001_023E":"Male, 75 to 79 years",
                    "B01001_024E":"Male, 80 to 84 years",
                    "B01001_025E":"Male, 85 years and over",
                    "B01001_027E":"Female, Under 5 years",
                    "B01001_028E":"Female, 5 to 9 years",
                    "B01001_029E":"Female, 10 to 14 years",
                    "B01001_030E":"Female, 15 to 17 years",
                    "B01001_031E":"Female, 18 and 19 years",
                    "B01001_032E":"Female, 20 years",
                    "B01001_033E":"Female, 21 years",
                    "B01001_034E":"Female, 22 to 24 years",
                    "B01001_035E":"Female, 25 to 29 years",
                    "B01001_036E":"Female, 30 to 34 years",
                    "B01001_037E":"Female, 35 to 39 years",
                    "B01001_038E":"Female, 40 to 44 years",
                    "B01001_039E":"Female, 45 to 49 years",
                    "B01001_040E":"Female, 50 to 54 years",
                    "B01001_041E":"Female, 55 to 59 years",
                    "B01001_042E":"Female, 60 and 61 years",
                    "B01001_043E":"Female, 62 to 64 years",
                    "B01001_044E":"Female, 65 and 66 years",
                    "B01001_045E":"Female, 67 to 69 years",
                    "B01001_046E":"Female, 70 to 74 years",
                    "B01001_047E":"Female, 75 to 79 years",
                    "B01001_048E":"Female, 80 to 84 years",
                    "B01001_049E":"Female, 85 years and over"}

In [6]:
demographic_vars = {'B02001_002E': 'White alone',
                    'B02001_003E': 'Black alone',
                    'B02001_005E': 'Asian alone',
                    'B02001_004E': 'Native American alone',
                    'B02001_007E': 'Other race alone',
                    'B03002_012E': 'Hispanic or Latino',
                    'B11001_007E': 'Non-family households',
                    'B11001_002E': 'Family households'}

In [7]:
education_vars = {"B15003_017E":"Regular high school diploma",
                  "B15003_018E":"GED or alternative credential",
                  "B15003_019E":"Some college, less than 1 year",
                  "B15003_020E":"Some college, 1 or more years,",
                  "B15003_021E":"Associate's degree",
                  "B15003_022E":"Bachelor's degree",
                  "B15003_023E":"Master's degree",
                  "B15003_024E":"Professional school degree",
                  "B15003_025E":"Doctorate degree"}

In [8]:
income_vars = {'B19001_002E':"Less than $10,000",
               'B19001_003E':"$10,000 to $14,999",
               'B19001_004E':"$15,000 to $19,999",
               'B19001_005E':"$20,000 to $24,999",
               'B19001_006E':"$25,000 to $29,999",
               'B19001_007E':"$30,000 to $34,999",
               'B19001_008E':"$35,000 to $39,999",
               'B19001_009E':"$40,000 to $44,999",
               'B19001_010E':"$45,000 to $49,999",
               'B19001_011E':"$50,000 to $59,999",
               'B19001_012E':"$60,000 to $74,999",
               'B19001_013E':"$75,000 to $99,999",
               'B19001_014E':"$100,000 to $124,999",
               'B19001_015E':"$125,000 to $149,999",
               'B19001_016E':"$150,000 to $199,999",
               'B19001_017E':"$200,000 or more",
               'B19013_001E': 'Median HH income'}

## 3. Function that runs GET request query for data from Census API

In [27]:
def get_acs_5y(year, variable_dict):
    
    state='36'

    county = county_list
    features = ','.join(variable_dict.keys())
    featurenames = variable_dict

    if year in [2015, 2016]: 
        url = 'https://api.census.gov/data/{}/acs/acs5?get=NAME,{}&for=block%20group:*&in=state:{}%20county:005&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef'.format(year,features,state)
    else: 
        url = 'https://api.census.gov/data/{}/acs5?get=NAME,{}&for=block%20group:*&in=state:{}%20county:005&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef'.format(year,features,state)
    print(url)
    r = requests.get(url)
    x = r.json()
    all_counties = pd.DataFrame(data=x[1:],columns=x[0])

    for i in county[1:]:
       
        if year in [2015, 2016]: 
            url = 'https://api.census.gov/data/{}/acs/acs5?get=NAME,{}&for=block%20group:*&in=state:{}%20county:{}&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef'.format(year,features,state,i)
        else: 
            url = 'https://api.census.gov/data/{}/acs5?get=NAME,{}&for=block%20group:*&in=state:{}%20county:{}&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef'.format(year,features,state,i)
#         print(url)
        r = requests.get(url)
        x = r.json()
        
        df = pd.DataFrame(data=x[1:],columns=x[0])
#         print (df.county.unique())
        all_counties = all_counties.append(df)

    all_counties_cleaned = all_counties.rename(columns={**{'GEO.idB01003':'GEO.ID',
                                                        'GEO.id2B01003':'GEO.ID2'},**featurenames
                                                       })


    all_counties_cleaned['GEOID']=all_counties_cleaned['state']+all_counties_cleaned['county']+all_counties_cleaned['tract']+all_counties_cleaned['block group']
    all_counties_cleaned=all_counties_cleaned.reset_index()
    return all_counties_cleaned

In [30]:
# make sure GEOIDs will match
blk_grp['GEOID']= blk_grp.apply(lambda x: '{}'.format(x['GEO_ID'][9:]), axis=1)

In [63]:
# Big data query! Get all the ACS data in dataframe, join to blk_grp shapefile, write to CARTO

for name, var_dict in {'demo':demographic_vars, 'age_sex':age_sex_vars, 'income':income_vars, 'edu':education_vars}.items():
    acs_2012 = get_acs_5y(2012, var_dict)
    acs_2013 = get_acs_5y(2013, var_dict)
    acs_2014 = get_acs_5y(2014, var_dict)
    acs_2015 = get_acs_5y(2015, var_dict)
    acs_2016 = get_acs_5y(2016, var_dict)
    
    acs_2016_blk_grp = blk_grp.merge(acs_2016,on='GEOID').to_crs({'init':'epsg:4326'})
    acs_2015_blk_grp = blk_grp.merge(acs_2015,on='GEOID').to_crs({'init':'epsg:4326'})
    acs_2014_blk_grp = blk_grp.merge(acs_2014,on='GEOID').to_crs({'init':'epsg:4326'})
    acs_2013_blk_grp = blk_grp.merge(acs_2013,on='GEOID').to_crs({'init':'epsg:4326'})
    acs_2012_blk_grp = blk_grp.merge(acs_2012,on='GEOID').to_crs({'init':'epsg:4326'})
    
    cc.write(acs_2012_blk_grp,'nyc_acs_{}_2012_blk_grp'.format(name), overwrite=True)
    cc.write(acs_2013_blk_grp,'nyc_acs_{}_2013_blk_grp'.format(name), overwrite=True)
    cc.write(acs_2014_blk_grp,'nyc_acs_{}_2014_blk_grp'.format(name), overwrite=True)
    cc.write(acs_2015_blk_grp,'nyc_acs_{}_2015_blk_grp'.format(name), overwrite=True)
    cc.write(acs_2016_blk_grp,'nyc_acs_{}_2016_blk_grp'.format(name), overwrite=True)

https://api.census.gov/data/2012/acs5?get=NAME,B02001_002E,B02001_003E,B02001_005E,B02001_004E,B02001_007E,B03002_012E,B11001_007E,B11001_002E&for=block%20group:*&in=state:36%20county:005&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef
https://api.census.gov/data/2013/acs5?get=NAME,B02001_002E,B02001_003E,B02001_005E,B02001_004E,B02001_007E,B03002_012E,B11001_007E,B11001_002E&for=block%20group:*&in=state:36%20county:005&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef
https://api.census.gov/data/2014/acs5?get=NAME,B02001_002E,B02001_003E,B02001_005E,B02001_004E,B02001_007E,B03002_012E,B11001_007E,B11001_002E&for=block%20group:*&in=state:36%20county:005&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef
https://api.census.gov/data/2015/acs/acs5?get=NAME,B02001_002E,B02001_003E,B02001_005E,B02001_004E,B02001_007E,B03002_012E,B11001_007E,B11001_002E&for=block%20group:*&in=state:36%20county:005&key=d04ebb69f2fc3fb6b720fdd9d37c97d2e6a7d7ef
https://api.census.gov/data/2016/acs/acs5?get=NAME,B02001_002E,B

/Users/michelleho/anaconda3/lib/python3.6/site-packages/carto/resources.py:90: FutureWarning: This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment
  warnings.warn('This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment', FutureWarning)


The following columns were changed in the CARTO copy of this dataframe:
GEO_ID -> geo_id
STATE -> state
COUNTY -> county
TRACT -> tract
BLKGRP -> blkgrp
NAME_x -> name_x
LSAD -> lsad
CENSUSAREA -> censusarea
GEOID -> geoid
NAME_y -> name_y
White alone -> white_alone
Black alone -> black_alone
Asian alone -> asian_alone
Native American alone -> native_american_alone
Other race alone -> other_race_alone
Hispanic or Latino -> hispanic_or_latino
Non-family households -> non_family_households
Family households -> family_households
block group -> block_group
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nyc_acs_demo_2012_blk_grp
The following columns were changed in the CARTO copy of this dataframe:
GEO_ID -> geo_id
STATE -> state
COUNTY -> county
TRACT -> tract
BLKGRP -> blkgrp
NAME_x -> name_x
LSAD -> lsad
CENSUSAREA -> censusarea
GEOID -> geoid
NAME_y -> name_y
White alone -> white_alone
Black alone -> black_alone
Asian alone -> asian_alone
Native American alone

In [70]:
# Median_hh_income has '' for empty/NULL values. Make sure it's null

for name, var_dict in {'income':income_vars}.items():
    for year in ['2012','2013','2014','2015','2016']:
        query = '''UPDATE nyc_acs_{name}_{year}_blk_grp SET median_hh_income = cast(nullif(median_hh_income, '') AS int);'''
        cc.query(query.format(name=name, year=year))

In [71]:
# To make sure all the data variables are integers.

alter_statement = 'ALTER COLUMN {col} TYPE INT USING {col}::integer'

for name, var_dict in {'demo':demographic_vars, 'age_sex':age_sex_vars, 'income':income_vars, 'edu':education_vars}.items():
    for year in ['2012','2013','2014','2015','2016']:
        columns_to_change = []
        for i, value in var_dict.items():
            statement = alter_statement.format(col=cartoframes.utils.norm_colname(value))
            columns_to_change.append(statement)
        columns_to_change = ', '.join(columns_to_change)
#         print(columns_to_change)

        make_numeric = '''ALTER TABLE nyc_acs_{name}_{year}_blk_grp {columns_to_change};
        '''
#         print(make_numeric.format(name=name, year=year, columns_to_change=columns_to_change))
        cc.query(make_numeric.format(name=name, year=year, columns_to_change=columns_to_change))

In [64]:
# Spatial join to NYNTA tables

alter_update_ntacode_join = '''
ALTER TABLE nyc_acs_{name}_{year}_blk_grp
ADD COLUMN ntacode text;

UPDATE nyc_acs_{name}_{year}_blk_grp
SET ntacode = b.ntacode
FROM nynta as b
WHERE st_intersects(nyc_acs_{name}_{year}_blk_grp.the_geom, b.the_geom);
'''
for name in {'demo':demographic_vars}.keys():
    for year in ['2012','2013','2014','2015','2016']:
        query = alter_update_ntacode_join.format(name=name, year=year)
#         print(query)
        cc.query(query)

In [87]:
# Big query that creates tables. Group by NTA and sum all variables (except for median_hh_income)

sum_query = '''
SELECT ntacode, {sum_statement}
FROM nyc_acs_{name}_{year}_blk_grp
GROUP BY ntacode
'''
sum_variable = 'sum({var}) as {var}'

for name, var_dict in {'demo':demographic_vars, 'age_sex':age_sex_vars, 'income':income_vars, 'edu':education_vars}.items():
    for year in ['2012','2013','2014','2015','2016']:
        sums = []
        for acs_varcode, var_name in var_dict.items():
            if var_name != 'Median HH income':
                sums.append(sum_variable.format(var=cartoframes.utils.norm_colname(var_name)))
        sum_statement = ','.join(sums)
        sum_query_formatted = sum_query.format(sum_statement=sum_statement, name=name, year=year)
#         print(sum_query_formatted)
        cc.query(sum_query_formatted,table_name='nta_acs_{name}_{year}'.format(name=name,year=year))


Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_demo_2012
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_demo_2013
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_demo_2014
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_demo_2015
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_demo_2016
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_age_sex_2012
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_age_sex_2013
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_age_sex_2014
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_age_sex_2015
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nta_acs_age_sex_2016
Table successfully written to CARTO: https://wxu-carto.carto.com/dataset/nt